In [1]:
import pandas as pd

file = "Resources/purchase_data.csv"

p_df = pd.read_csv(file)
p_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [2]:
p_df.rename(mapper = lambda i: i.replace(' ','').lower(), axis = 1, inplace = True)
p_df

,purchaseid,sn,age,gender,itemid,itemname,price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [3]:
###Put results in dataframes
###Use unique to get total number of players
pd.DataFrame([p_df["sn"].nunique()], columns = ["Total Players"])

,Total Players
0,576


In [4]:
#items.nunique
pd.DataFrame([[p_df["itemname"].nunique(), "${price:.2f}".format(price = p_df["price"].mean()), \
               p_df["purchaseid"].count(),"${price:.2f}".format(price = p_df["price"].sum())]], \
               columns = ["Number of Unique", "Items Average Price", "Number of Purchases", "Total Revenue"])

,Number of Unique,Items Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [5]:
#reduce df by sn
p2_df = p_df.drop_duplicates(subset=["sn"])
gender_df = p2_df["gender"].value_counts().to_frame()
gender_df["Percentage of Players"] = gender_df["gender"]/gender_df["gender"].sum()
gender_df["Percentage of Players"] = gender_df["Percentage of Players"].apply(lambda x : x * 100).map("{:.2f}%".format)
gender_df.rename(columns = {"gender" : "Gender"})

,Gender,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [6]:
p1 = p_df[["gender","sn"]].groupby("gender").count()
p2 = p_df[["gender","price"]].groupby("gender").sum()
genderanalysis = pd.merge(p1, p2, on = "gender")
genderanalysis["Average Purchase Price"] = genderanalysis["price"]/genderanalysis["sn"]
genderanalysis["Avg Total Purchase per Person"] = genderanalysis["price"]/gender_df["gender"]
genderanalysis.rename(columns = {"price":"Total Purchase Value", "sn" : "Purchase Count"}, inplace = True)
genderanalysis.index.rename(name = "Gender", inplace = True)
for i in genderanalysis.columns[1:]:
    genderanalysis[i] = genderanalysis[i].map("${:.2f}".format)
genderanalysis



,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,$1967.64,$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [7]:
bins = [5*i - 1 for i in range(2,9)]
groupnames = [str(i + 1)+'-'+str(i + 5) for i in bins[:-1]]
bins.insert(0,0)
bins.append(100)
groupnames.insert(0, "<10")
groupnames.append(">40")
pages_df = p_df.copy()
pages_df["age range"] = pd.cut(pages_df["age"], bins, labels = groupnames, include_lowest = False)
pages_df

,purchaseid,sn,age,gender,itemid,itemname,price,age range
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,>40
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24
778,778,Sisur91,7,Male,92,Final Critic,4.19,<10


In [8]:
agecount_df = pages_df[["age range","sn"]].drop_duplicates(subset=["sn"]).groupby("age range").count().rename(columns = {"sn":"Total Count"})
agecount_df
agecount_df["Percentage of Players"] = agecount_df["Total Count"]/agecount_df["Total Count"].sum()
agecount_df.index.rename(None, inplace = True)
agecount_df["Percentage of Players"] = agecount_df["Percentage of Players"].apply(lambda x : x * 100).map("{:.2f}%".format)
agecount_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
>40,12,2.08%


In [9]:
p1 = pages_df[["age range","sn"]].groupby("age range").count()
p2 = pages_df[["age range","price"]].groupby("age range").sum()
p3 = pages_df[["age range","sn"]].groupby("age range").nunique()
ageanalysis = pd.merge(p1, p2, on = "age range")
ageanalysis["Average Purchase Price"] = ageanalysis["price"]/ageanalysis["sn"]
ageanalysis["Avg Total Purchase per Person"] = ageanalysis["price"]/p3["sn"]
ageanalysis.rename(columns = {"price":"Total Purchase Value", "sn" : "Purchase Count"}, inplace=True)
ageanalysis.index.rename(name = "Age Range", inplace = True)
for i in ageanalysis.columns[1:]:
    ageanalysis[i] = ageanalysis[i].map("${:.2f}".format)
ageanalysis


,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Age Range,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,$1114.06,$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
>40,13,$38.24,$2.94,$3.19


In [10]:
topspenders = pd.merge(p_df[["sn","price"]].groupby("sn").sum().sort_values("price", ascending = False).head(5), \
                       p_df[["sn","price"]].groupby("sn").count(), \
                       on = "sn")
topspenders["Average Purchase Value"] = topspenders["price_x"]/topspenders["price_y"]
topspenders = topspenders[["price_y","Average Purchase Value","price_x"]] \
                         .rename(columns = {"price_x" : "Average Purchase Price", "price_y" : "Purchase Count"})
topspenders.index.rename("SN", inplace=True)
for i in topspenders.columns[1:]:
    topspenders[i] = topspenders[i].map("${:.2f}".format)
topspenders

,Purchase Count,Average Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [13]:
def itemsummary(x):
        d = []
        a = x['price'].count()
        b = x['price'].sum()
        d.append(a)
        d.append("${x:.2f}".format(x = b/a))
        d.append("${x:.2f}".format(x = b))
        return pd.Series(d, index=['Purchase Count', 'Item Price', 'Total Purchase Value'])


t = p_df[['itemid', 'itemname','price']].groupby(['itemid','itemname']).apply(itemsummary)
t.sort_values(by = "Purchase Count", ascending = False).head(10)

,,Purchase Count,Item Price,Total Purchase Value
itemid,itemname,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
75,Brutality Ivory Warmace,8,$2.42,$19.36
103,Singed Scalpel,8,$4.35,$34.80
34,Retribution Axe,8,$2.22,$17.76


In [35]:
pd.merge(t, (t["Total Purchase Value"].map(lambda x: float(x[1:]))).rename("float"), on = 'itemid') \
    .sort_values(by = "float", ascending = False) \
    .drop("float", axis = 1)

,Purchase Count,Item Price,Total Purchase Value
itemid,,,
92,13,$4.61,$59.99
178,12,$4.23,$50.76
82,9,$4.90,$44.10
145,9,$4.58,$41.22
103,8,$4.35,$34.80
...,...,...,...
28,2,$1.06,$2.12
125,2,$1.00,$2.00
126,1,$2.00,$2.00
